In [1]:
from random import random
from struct import unpack
from os import urandom as urandom
from collections import defaultdict
from abc import ABC, abstractmethod
from typing import Dict, Tuple, Optional

In [2]:
#
# clone this repo: https://github.com/stevenang/randomness_testsuite
# and put this file into the folder for the below imports to work..
#
from FrequencyTest import FrequencyTest
from RunTest import RunTest
from Matrix import Matrix
from Spectral import SpectralTest
from TemplateMatching import TemplateMatching
from Universal import Universal
from Complexity import ComplexityTest
from Serial import Serial
from ApproximateEntropy import ApproximateEntropy
from CumulativeSum import CumulativeSums
from RandomExcursions import RandomExcursions

In [3]:
def print_bits(input_bit, output_bit):
    pass
    #print(input_bit, output_bit if output_bit is not None else '.')


class RandomGenerator:
    def __init__(self):
        self.bit_cache = []
        self.byte_cache = []
        self.cache_size = 128  # Number of bytes to retrieve at a time

    def _fill_bit_cache(self):
        random_bytes = urandom(self.cache_size)
        self.bit_cache = [
            (byte >> i) & 1
            for byte in random_bytes
            for i in range(8)
        ]

    def _fill_byte_cache(self):
        self.byte_cache = list(urandom(self.cache_size))

    def random_bit(self) -> int:
        if not self.bit_cache:
            self._fill_bit_cache()
        return self.bit_cache.pop(0)

    def random_float(self) -> float:
        if not self.byte_cache:
            self._fill_byte_cache()
        random_bytes = bytes(self.byte_cache[:8])
        self.byte_cache = self.byte_cache[8:]
        random_float = unpack('>Q', random_bytes)[0] / (1 << 64)
        return random_float


rg = RandomGenerator()

def random_bit() -> int:
    return rg.random_bit()

def random_float() -> float:
    return rg.random_float()


class BitSource(ABC):
    @abstractmethod
    def next(self) -> int:
        pass


class RandomnessExtractor(ABC):
    @abstractmethod
    def next(self) -> Optional[int]:
        pass

    def random(self) -> int:
        result = None
        while result is None:
            result = self.next()
        return result


class BiasedBitSource(BitSource):
    def __init__(self, bias=None):
        if bias is None:
            bias = random_float()
        self.bias = bias

    def next(self) -> int:
        if random_float() < self.bias:
            return 0
        else:
            return 1


class VaryingBiasedBitSource(BitSource):
    def __init__(self, min_cutoff=0, max_cutoff=1):
        self.min_cutoff = min_cutoff
        self.max_cutoff = max_cutoff
        self.bias = random_float()

    def next_bias(self, draw):
        bias = draw**2
        if random_bit() == 0:
            bias = 1 - bias
        bias = max(self.min_cutoff, bias)
        bias = min(self.max_cutoff, bias)
        return bias

    def next(self) -> int:
        draw = random_float()
        if draw < self.bias:
            result = 0
        else:
            result = 1
        self.bias = self.next_bias(draw)
        return result


class PassthroughExtractor(RandomnessExtractor):
    def __init__(self, bit_source: BitSource):
        self.bit_source = bit_source
        self.bits = []

    def next(self) -> Optional[int]:
        bit = self.bit_source.next()
        result = bit
        print_bits(bit, result)
        return result


class VonNeumannExtractor(RandomnessExtractor):
    def __init__(self, bit_source: BitSource):
        self.bit_source = bit_source
        self.bits = []

    def next(self) -> Optional[int]:
        bit = self.bit_source.next()
        if bit not in [0, 1]:
            raise ValueError("Bit must be 0 or 1.")
        self.bits.append(bit)
        result = None
        if len(self.bits) == 2:
            if self.bits == [0, 1]:
                result = 1
            elif self.bits == [1, 0]:
                result = 0
            self.bits = []  # Reset for the next pair of bits
        print_bits(bit, result)
        return result


class ParityExtractor(RandomnessExtractor):
    def __init__(self, bit_source: BitSource, N: int):
        self.bit_source = bit_source
        if N <= 0:
            raise ValueError("N must be a positive integer.")
        self.N = N
        self.bits = []

    def next(self) -> Optional[int]:
        bit = self.bit_source.next()
        if bit not in [0, 1]:
            raise ValueError("Bit must be 0 or 1.")
        self.bits.append(bit)
        result = None
        if len(self.bits) == self.N:
            result = sum(self.bits) % 2
            self.bits = []  # Reset for the next block of N bits
        print_bits(bit, result)
        return result


class MarkovChainBitSource(BitSource):
    def __init__(self, start_state: str):
        self.current_state = start_state
    
    @abstractmethod
    def get_transitions(self, state: str) -> Dict[int, Tuple[str, float]]:
        pass

    def next(self) -> int:
        transitions = self.get_transitions(self.current_state)
        rand = random_float()
        cumulative_probability = 0.0
        for bit, (next_state, probability) in transitions.items():
            cumulative_probability += probability
            if rand < cumulative_probability:
                self.current_state = next_state
                return bit
        raise RuntimeError("Probabilities do not sum to 1.")


class BlumExtractor(RandomnessExtractor):
    def __init__(self, bit_source: MarkovChainBitSource):
        self.bit_source = bit_source
        self.state_bits = defaultdict(list)  # Stores the last bits for each state

    def next(self) -> Optional[int]:
        current_state = self.bit_source.current_state
        bit = self.bit_source.next()
        self.state_bits[current_state].append(bit)
        result = None
        if len(self.state_bits[current_state]) == 2:
            bits = self.state_bits[current_state]
            if bits == [0, 1]:
                result = 1
            elif bits == [1, 0]:
                result = 0
            self.state_bits[current_state] = []
        print_bits(bit, result)
        return result

class ExampleMarkovChainBitSource(MarkovChainBitSource):
    def __init__(self):
        super().__init__(start_state='A')
        self.transitions = {
            'A': {0: ('B', 0.5), 1: ('C', 0.5)},
            'B': {0: ('A', 0.7), 1: ('C', 0.3)},
            'C': {0: ('A', 0.4), 1: ('B', 0.6)},
        }
    
    def get_transitions(self, state: str) -> Dict[int, Tuple[str, float]]:
        return self.transitions[state]

In [4]:
def run_tests(r):
    funcs = [
        FrequencyTest.monobit_test,
        FrequencyTest.block_frequency,
        RunTest.run_test,
        RunTest.longest_one_block_test,
        Matrix.binary_matrix_rank_text,
        SpectralTest.spectral_test,
        TemplateMatching.non_overlapping_test,
        TemplateMatching.overlapping_patterns,
        Universal.statistical_test,
        ComplexityTest.linear_complexity_test,
        lambda r: Serial.serial_test(r)[0] and Serial.serial_test(r)[1],
        ApproximateEntropy.approximate_entropy_test,
        lambda r: CumulativeSums.cumulative_sums_test(r, 0),
        lambda r: CumulativeSums.cumulative_sums_test(r, 1),
        #RandomExcursions.random_excursions_test,
        #RandomExcursions.variant_test,
    ]
    num_passed = 0
    for f in funcs:
        result = f(r)
        #print(result)
        num_passed += int(result[1])
    passed = (num_passed >= 10)
    if passed:
        print(f'PASSED: The bit sequence appears to be a sequence of independent fair coin tosses ({num_passed}/{len(funcs)} passed) ✅')
    else:
        print(f'FAILED: The bit sequence does not appear to be a sequence of independent fair coin tosses ({num_passed}/{len(funcs)} passed) ❌')

In [5]:
N = 8*1024

In [6]:
r = ''.join(['0' for _ in range(N)])
run_tests(r)

FAILED: The bit sequence does not appear to be a sequence of independent fair coin tosses (2/14 passed) ❌


In [7]:
r = ''.join([str(round(random())) for _ in range(N)])
run_tests(r)

PASSED: The bit sequence appears to be a sequence of independent fair coin tosses (12/14 passed) ✅


In [8]:
r = ''.join([str(random_bit()) for _ in range(N)])
run_tests(r)

PASSED: The bit sequence appears to be a sequence of independent fair coin tosses (13/14 passed) ✅


In [9]:
bit_source = BiasedBitSource(0.9)

r = ''.join([str(bit_source.next()) for _ in range(N)])
run_tests(r)

FAILED: The bit sequence does not appear to be a sequence of independent fair coin tosses (3/14 passed) ❌


In [10]:
bit_source = BiasedBitSource(0.9)
extractor = VonNeumannExtractor(bit_source)

r = ''.join([str(extractor.random()) for _ in range(N)])
run_tests(r)

PASSED: The bit sequence appears to be a sequence of independent fair coin tosses (13/14 passed) ✅


In [11]:
bit_source = VaryingBiasedBitSource(min_cutoff=0.1, max_cutoff=0.7)

r = ''.join([str(bit_source.next()) for _ in range(N)])
run_tests(r)

FAILED: The bit sequence does not appear to be a sequence of independent fair coin tosses (7/14 passed) ❌


In [12]:
bit_source = VaryingBiasedBitSource(min_cutoff=0.1, max_cutoff=0.7)
extractor = ParityExtractor(bit_source=bit_source, N=4)

r = ''.join([str(extractor.random()) for _ in range(N)])
run_tests(r)

PASSED: The bit sequence appears to be a sequence of independent fair coin tosses (13/14 passed) ✅


In [13]:
bit_source = ExampleMarkovChainBitSource()

r = ''.join([str(bit_source.next()) for _ in range(N)])
run_tests(r)

FAILED: The bit sequence does not appear to be a sequence of independent fair coin tosses (5/14 passed) ❌


In [14]:
bit_source = ExampleMarkovChainBitSource()
extractor = VonNeumannExtractor(bit_source)

r = ''.join([str(extractor.random()) for _ in range(N)])
run_tests(r)

PASSED: The bit sequence appears to be a sequence of independent fair coin tosses (10/14 passed) ✅


In [15]:
bit_source = ExampleMarkovChainBitSource()
extractor = ParityExtractor(bit_source=bit_source, N=8)

r = ''.join([str(extractor.random()) for _ in range(N)])
run_tests(r)

PASSED: The bit sequence appears to be a sequence of independent fair coin tosses (13/14 passed) ✅


In [16]:
bit_source = ExampleMarkovChainBitSource()
extractor = BlumExtractor(bit_source)

r = ''.join([str(extractor.random()) for _ in range(N)])
run_tests(r)

PASSED: The bit sequence appears to be a sequence of independent fair coin tosses (12/14 passed) ✅


In [17]:
class ExampleMarkovChainBitSource(MarkovChainBitSource):
    def __init__(self):
        super().__init__(start_state='A')
        self.transitions = {
            'A': {0: ('A', 0.99), 1: ('B', 0.01)},
            'B': {0: ('A', 0.01), 1: ('B', 0.99)},
        }
        
    def get_transitions(self, state: str) -> Dict[int, Tuple[str, float]]:
        return self.transitions[state]

In [18]:
bit_source = ExampleMarkovChainBitSource()

r = ''.join([str(bit_source.next()) for _ in range(N)])
run_tests(r)

FAILED: The bit sequence does not appear to be a sequence of independent fair coin tosses (0/14 passed) ❌


In [19]:
bit_source = ExampleMarkovChainBitSource()
extractor = VonNeumannExtractor(bit_source)

r = ''.join([str(extractor.random()) for _ in range(N)])
run_tests(r)

FAILED: The bit sequence does not appear to be a sequence of independent fair coin tosses (9/14 passed) ❌


In [20]:
bit_source = ExampleMarkovChainBitSource()
extractor = ParityExtractor(bit_source=bit_source, N=32)

r = ''.join([str(extractor.random()) for _ in range(N)])
run_tests(r)

FAILED: The bit sequence does not appear to be a sequence of independent fair coin tosses (3/14 passed) ❌


In [21]:
bit_source = ExampleMarkovChainBitSource()
extractor = BlumExtractor(bit_source)

r = ''.join([str(extractor.random()) for _ in range(N)])
run_tests(r)

PASSED: The bit sequence appears to be a sequence of independent fair coin tosses (13/14 passed) ✅
